In [2]:
import pandas as pd
import numpy as np
import os
import pymysql

In [3]:
# 1. connect
conn = pymysql.connect(host="127.0.0.1", port=3306, db='finalteam3', user="kdigital", password="mysql", charset="utf8")

# 2. get command object
cursor = conn.cursor()

# 3. execute command
sql = """select m.movie_id, vote_average, vote_count, popularity,title, mg.genre_id, g.name, m.posterpath
         from movie m
         join movie_genre mg on (m.movie_id = mg.movie_id)
         join genre g on(g.genre_id = mg.genre_id)
         where vote_count > 3"""
cursor.execute(sql)

# 4. close resource
cursor.close()
conn.close()

In [4]:
rows = cursor.fetchall()
movies = pd.DataFrame(rows, columns=['movie_id','vote_average','vote_count','popularity','title','genre_id','genre','posterpath'])

In [5]:
data = movies[['movie_id','genre','vote_count','title','vote_average','posterpath']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111337 entries, 0 to 111336
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movie_id      111337 non-null  int64  
 1   genre         111337 non-null  object 
 2   vote_count    111337 non-null  int64  
 3   title         111337 non-null  object 
 4   vote_average  111337 non-null  float64
 5   posterpath    111337 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 5.1+ MB


In [6]:
data["genre"] = data.groupby(by=["movie_id", "title", "vote_count", "vote_average"])["genre"].transform(lambda row: ','.join(row))
data.drop_duplicates(inplace=True)
data['genre'] = data['genre'].map(lambda sc : ','.join(np.unique(sc.split(','))))
data.shape

/Users/parkseonghun/opt/miniconda3/envs/kdigital-busan/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/parkseonghun/opt/miniconda3/envs/kdigital-busan/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/parkseonghun/opt/miniconda3/envs/kdigital-busan/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

(58389, 6)

In [7]:
data = data.reset_index(drop=True)

In [8]:
data

,movie_id,genre,vote_count,title,vote_average,posterpath
0,11,"SF,모험,액션",16272,스타워즈: 에피소드 4 새로운 희망,8.2,/7XFfURIFCJxN1mfBg0SAjk5yGzg.jpg
1,18,"SF,모험,스릴러,액션,판타지",8583,제5원소,7.5,/npcHgyp5MmKMe4lWqJi2ma1CuhC.jpg
2,22,"모험,액션,판타지",16645,캐리비안의 해적: 블랙펄의 저주,7.8,/3ovejwQO4fKOWx4VgGBJeT8CKCn.jpg
3,58,"모험,액션,판타지",12772,캐리비안의 해적: 망자의 함,7.3,/6tD4oT7C01aZLVfoZDz5VFVbJCi.jpg
4,62,"SF,모험,미스터리",8862,2001 스페이스 오디세이,8.1,/fTzcVo738Ys88FPVG987TH4aJKt.jpg
...,...,...,...,...,...,...
58384,605045,TV 영화,9,Am I a Serial Killer?,6.2,/zycNjyjasWMK28PEfv6giK6oH1S.jpg
58385,632530,TV 영화,6,Return to Downton Abbey: A Grand Event,7.7,/5glPZtRz7KxpfMcohhlN7sRF1O1.jpg
58386,680989,TV 영화,6,Tutto il giorno davanti,6.5,/2uiSBsqv213EelC4ihIt2GZHC8s.jpg
58387,744945,TV 영화,8,Spotlight on Christmas,6.6,/n7VOErjeOwTz0YMiODDBUU74bTw.jpg


In [9]:
m = data['vote_count'].quantile(0)
print(m)
data = data.loc[data['vote_count']>= m]

4.0


In [10]:
data.shape

(58389, 6)

In [11]:
c = data['vote_average'].mean()
print(c)

6.119085786706401


In [12]:
def weight_rating(x, m=m, c=c):
    v = x['vote_count']
    r = x['vote_average']

    return( v / (v+m) *r) + (m/(v+m)*c)

In [13]:
data['score'] = data.apply(weight_rating, axis=1)

In [82]:
data[data['title'] == '블랙 위도우']

,movie_id,genre,vote_count,title,vote_average,score
2677,497698,"SF,모험,액션",6189,블랙 위도우,7.6,7.598652
35562,19345,"미스터리,스릴러",137,블랙 위도우,6.1,6.104522


In [14]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [15]:
counter = CountVectorizer(ngram_range=(1,3))

In [16]:
data['genre']=data['genre'].astype(str)

In [17]:
c_count_genres = counter.fit_transform(data['genre'])

In [18]:
tfidf_transformer = TfidfTransformer()

In [19]:
c_t_genres = tfidf_transformer.fit_transform(c_count_genres)

In [20]:
c_t_genres

<58389x880 sparse matrix of type '<class 'numpy.float64'>'
	with 192709 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_genre = cosine_similarity(c_t_genres, c_t_genres)
print(similarity_genre)

[[1.         0.3275978  0.34661957 ... 0.         0.         0.        ]
 [0.3275978  1.         0.32077943 ... 0.         0.         0.        ]
 [0.34661957 0.32077943 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 1.         1.         1.        ]]


In [25]:
similarity_genre=similarity_genre.argsort()[:,::-1]

In [26]:
similarity_genre.dtype

dtype('int64')

In [27]:
def recommend_movie_list(df, movie_title, top=30):
    # 특정영화 정보 뽑아내기
    target_movie_index = df[df['movie_id'] == movie_title].index.values

    # 타겟영화와 비슷한 코사인 유사도값
    sim_index = similarity_genre[target_movie_index, :(top*2)].reshape(-1,1)
    # 본인제외
    sim_index = sim_index[sim_index != target_movie_index ]

    # 추천결과 새로운 df생성, 평균평점(score)으로 정렬
    result = df.iloc[sim_index].sort_values('score', ascending=False)[:top]

    return result

In [28]:
recommend_movie_list(data, 129)

,movie_id,genre,vote_count,title,vote_average,posterpath,score
3964,8392,"가족,애니메이션,판타지",5758,이웃집 토토로,8.1,/csiklRMzvXRLKtFcIlEDeEU7nWr.jpg,8.098625
5090,308141,"가족,애니메이션,판타지",9,Sofia the First: The Curse of Princess Ivy,8.9,/nre6FTGVmb4fnsWgzJfjDnI0omN.jpg,8.044334
4001,9479,"가족,애니메이션,판타지",7136,팀버튼의 크리스마스 악몽,7.8,/aWUy907W9OtypP3iWO3wWvu4641.jpg,7.799058
4215,14836,"가족,애니메이션,판타지",5378,코렐라인: 비밀의 문,7.8,/iGp4wBc0ZBVFd2HPicyDgl6Pn1r.jpg,7.798751
4447,167666,"가족,애니메이션,판타지",66,Monster High: Escape from Skull Shores,7.9,/ddyQjIlKdBtw7kWQ6lZofxOCIJU.jpg,7.798233
4127,12429,"가족,애니메이션,판타지",2999,벼랑 위의 포뇨,7.7,/gDT5xMYTLCJdLaUMJTqqfEfvSsD.jpg,7.697894
5873,708352,"가족,애니메이션,판타지",36,매직 스쿨: 초보마법사 아우라의 대모험,7.8,/9B10tJ9LrvzcYaMQdx6yyovszzN.jpg,7.631909
5535,505832,"가족,애니메이션,판타지",36,마이 리틀 포니: 이퀘스트리아 걸스 - 잊혀진 우정,7.8,/dY7pdaPfbTnJnfnZBjRFprgRuUn.jpg,7.631909
5553,514754,"가족,애니메이션,판타지",855,바오,7.6,/2KGopAhzzRzu7OENSWp8hqKTlnT.jpg,7.593104
4033,10144,"가족,애니메이션,판타지",6171,인어공주,7.4,/87RIt2AnjMe3QhxzhusolKJGgS2.jpg,7.399170


In [46]:
import pickle
with open('similarity_genre2.pickle', "wb") as f:
        pickle.dump(similarity_genre2, f, protocol=4)

In [28]:

with open('../flask-api-server/modules/similarity_genre.pickle', "wb") as f:
        similarity_genre = pickle.load(f)



In [29]:
type(similarity_genre)

numpy.ndarray

In [30]:
similarity_genre[0,0]

0

In [31]:
with open('../flask-api-server/modules/movie_data.pickle', "rb") as f:
    data = pickle.load(f)

In [32]:
data.shape

(42022, 6)

In [33]:
data.index

Int64Index([    0,     3,     5,     7,    10,    11,    12,    15,    17,
               20,
            ...
            84552, 84553, 84555, 84557, 84559, 84561, 84562, 84563, 84564,
            84566],
           dtype='int64', length=42022)

In [34]:
data.head()

,movie_id,genre,vote_count,title,vote_average,score
0,2,"드라마,범죄,코미디",150,아리엘,6.9,6.873374
3,3,"드라마,코미디",149,천국의 그림자,7.2,7.161589
5,5,"범죄,코미디",2035,포룸,5.7,5.701493
7,6,"범죄,스릴러,액션",215,킬러 나이트,6.5,6.492065
10,8,다큐멘터리,18,Life in Loops (A Megacities RMX),7.5,7.176931


In [35]:
data = data.reset_index(drop=True)

In [36]:
target_movie_index = data[data['movie_id'] == 363088].index.values
target_movie_index

array([28787])